<a href="https://colab.research.google.com/github/Pra123thyu/IIIT-FMML-ASSIGNMENT/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?**
Increasing the Percentage of the Validation Set:

Pros:

More Reliable Evaluation: A larger validation set provides a more reliable estimate of your model's performance. It reduces the variance in your evaluation metrics, making them less sensitive to random variations in the data.
Better Generalization Assessment: With a larger validation set, you get a better sense of how well your model generalizes to unseen data, which is the primary goal in machine learning.
Cons:

Smaller Training Set: As you allocate more data to the validation set, you'll have less data available for training your model. This can lead to a less effective model, especially if your dataset is small to begin with.
Slower Training: With a smaller training set, your model may converge more slowly or might require more epochs to reach a satisfactory performance level.
Impact on Accuracy: Initially, increasing the validation set size may lead to a more accurate evaluation of your model's performance. However, if you allocate too much data to the validation set, your model's training performance may suffer, leading to decreased accuracy in the long run.

Reducing the Percentage of the Validation Set:

Pros:

Larger Training Set: A smaller validation set means you have more data available for training your model. This can lead to a more powerful and accurate model.
Cons:

Less Reliable Evaluation: A smaller validation set can result in more variability in your evaluation metrics. The performance of your model on this set may not be as representative of its generalization performance.
Risk of Overfitting: With less data allocated for validation, there's a higher risk of overfitting. Your model might perform well on the validation set but poorly on unseen data.
Impact on Accuracy: Initially, reducing the validation set size may lead to an increase in training accuracy, but it can also make it difficult to accurately assess your model's true generalization performance. Overfitting is a significant concern when you have a small validation set

**How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?**
Larger Training Set:

When you have a larger training set, your model has more data to learn from. This can help it capture underlying patterns and relationships in the data better, leading to a potentially more accurate model.

With a large enough training set, the model is more likely to generalize well to unseen data, including the test set. This is because it has had exposure to a diverse range of examples during training.

If the training set is large enough and representative of the data distribution, the validation set's performance can serve as a good estimate of how the model will perform on the test set. This is because the validation set should also be a good representation of the data distribution.

Larger Validation Set:

A larger validation set can provide a more reliable estimate of your model's performance. It reduces the impact of random variations and noise in the validation results.

A larger validation set can better detect overfitting. When a model is overfitting (fitting the training data noise), it may perform very well on a small validation set due to chance. A larger validation set makes it more likely that the model's performance reflects its actual ability to generalize.

Balancing Train and Validation Set Sizes:

There is a trade-off between the size of the training and validation sets. Allocating too much data to the validation set reduces the amount of data available for training, potentially leading to underfitting, where the model doesn't learn the underlying patterns well.

Conversely, allocating too little data to the validation set can result in a noisy estimate of model performance and may not effectively detect overfitting.

In practice, it's common to use techniques like k-fold cross-validation to strike a balance between training and validation set sizes while ensuring robust model evaluation. In k-fold cross-validation, the data is divided into k subsets (folds), and the model is trained and validated k times, with each fold serving as the validation set once. This provides a more comprehensive assessment of your model's performance across different data subsets and helps mitigate the impact of a particular split's size on performance estimation.

Overall, the goal is to have a sufficient amount of data in both the training and validation sets to train a good model and accurately estimate its performance on unseen data (the test set). The specific size of these sets depends on your dataset, the problem at hand, and the resources available for experimentation.






**What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?**
Hold-Out Validation Split:

A common starting point is to reserve 70-80% of your data for training and the remaining 20-30% for validation. This is a reasonable split for many datasets and models.
k-Fold Cross-Validation:

If your dataset is relatively small, or if you want a more robust estimate of model performance, you can use k-fold cross-validation. Common values for k include 5 or 10, but you can adjust it based on your specific needs. In k-fold cross-validation, you divide your data into k subsets (folds) and perform training and validation k times, rotating which fold serves as the validation set in each iteration.
Stratified Sampling:

If your dataset has class imbalance (unequal distribution of classes), it's important to use stratified sampling when splitting your data into training and validation sets. This ensures that both sets maintain the same class distribution as the original dataset.
Leave-One-Out Cross-Validation (LOOCV):

In LOOCV, you train on all but one data point and validate on the single omitted point, repeating this process for every data point in your dataset. While this provides a robust estimate of performance, it can be computationally expensive and may not be practical for large datasets.
Adaptive Splits:

You can also adapt the train-validation split based on the size of your dataset. For very large datasets, you might allocate a smaller percentage to validation because you still have a substantial amount of data for training. Conversely, for small datasets, you might allocate a larger percentage to validation to ensure a robust performance estimate.
In practice, it's a good idea to experiment with different validation set sizes and techniques to find the one that works best for your specific problem. You can monitor how your model performs on the validation set as you adjust the split percentage to strike the right balance between accurate performance estimation and effective model training. Keep in mind that the ultimate goal is to evaluate your model's ability to generalize to unseen data, so a validation set that is representative of the data distribution is crucial.






## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**Does averaging the validation accuracy across multiple splits give more consistent results?**
Reduced Variance: Averaging over multiple splits helps reduce the impact of randomness or variability in a single split. Data points in a single validation set might be outliers or may not be representative of the overall data distribution. By averaging over multiple splits, you get a more stable and less biased estimate of your model's performance.

Robustness to Data Split: The choice of how to split the data into training and validation sets can influence your results. By using different splits, you ensure that your model's performance is evaluated across various subsets of the data, making the assessment more robust.

Better Generalization Estimate: Cross-validation provides a more accurate estimate of how well your model is likely to generalize to unseen data. It simulates the process of training and testing your model on multiple different subsets of the data, which is closer to the real-world scenario where the model encounters various data distributions.

Model Selection: If you're comparing multiple models or hyperparameter configurations, cross-validation allows you to compare their performance more reliably. Averaging over multiple splits helps you make more informed decisions about which model or configuration to choose.

Mitigating Data Imbalance: Cross-validation can help balance the effects of class imbalance in your dataset. In each fold, the data is partitioned such that it maintains the same class distribution as the original dataset, ensuring that each class is adequately represented in both training and validation sets in different folds.

However, it's important to note that k-fold cross-validation or any form of cross-validation can be computationally expensive, especially for large datasets or complex models. Additionally, the choice of the number of folds (k) can affect the trade-off between computational cost and the robustness of the estimate. Smaller values of k (e.g., 5 or 10) are common choices, but you can adjust it based on your specific needs and resources.

In summary, averaging validation accuracy across multiple splits through techniques like k-fold cross-validation is a valuable practice for obtaining more consistent and reliable estimates of your model's performance, especially when dealing with limited data or when you need a thorough assessment of generalization.






**Does it give more accurate estimate of test accuracy?**
Better Representation of Data Variability: Cross-validation simulates the process of training and testing your model on multiple different subsets of the data. This helps account for the variability in your dataset. If you use a single fixed validation split, it might by chance include data points that are either particularly easy or difficult to predict, leading to an overestimation or underestimation of model performance.

Reduces Overfitting Bias: With a single fixed validation split, there's a risk of overfitting to that specific split. Your model might learn to perform well on the validation set through repeated adjustments to the model's hyperparameters, but this doesn't necessarily indicate good generalization. Cross-validation mitigates this bias by assessing performance on multiple validation sets, making it harder for the model to overfit to any one of them.

Robustness to Data Split: Different data splits can influence model performance. Cross-validation allows you to evaluate the model's performance across various subsets of the data, which provides a more robust and representative estimate of how the model is likely to perform on unseen data.

**What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**
Training Iterations and Model Training:

Increasing the number of training iterations can improve the performance of your model during training. With more iterations, the model has more opportunities to adjust its weights and biases to better fit the training data.

However, there is a point of diminishing returns. After a certain number of iterations, the model may start to overfit the training data, meaning it becomes too specialized in capturing noise in the data rather than general patterns. This can lead to a decrease in performance on unseen data, including the validation and test sets.

The optimal number of training iterations often depends on factors such as the complexity of the model, the size of the dataset, and the learning rate. It's common to monitor the model's performance on the validation set during training and stop when performance plateaus or starts to degrade (early stopping).

Effect on Cross-Validation:

When using cross-validation to estimate model performance, the number of iterations during each fold can impact the estimate. More iterations can lead to a more finely tuned model, but it may also make each fold more time-consuming.

Cross-validation can be computationally expensive, especially with a large number of iterations per fold. You need to balance computational resources with the need for a reliable performance estimate.

In k-fold cross-validation, you typically train and validate the model k times, and each training iteration within a fold contributes to the model's overall performance estimate. Increasing the number of iterations per fold can make the estimate more robust, but it also increases the computational cost.

In summary, the number of training iterations during model training can impact how well the model fits the training data and, to some extent, its ability to generalize to unseen data. When using cross-validation to estimate model performance, more iterations per fold can lead to a more reliable estimate but at the cost of increased computation time. The optimal number of iterations depends on factors like the model's complexity, dataset size, and available resources, and it often involves a balance between achieving good training performance and avoiding overfitting.






**Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**

Increasing the number of iterations during training can help to some extent when dealing with a very small training dataset, but it's not a guaranteed solution, and there are limitations to what additional iterations can achieve. Here are some considerations:

Advantages of Increasing Iterations with a Small Training Dataset:

Improved Fit to Training Data: More iterations allow the model to continue adjusting its weights and biases to better fit the available training data. This can help the model capture more complex patterns and potentially improve training accuracy.

Mitigation of Underfitting: In cases where the model has not yet converged and is underfitting the training data, increasing iterations may help the model reach a better fit.

Limitations and Considerations:

Overfitting Risk: Increasing the number of iterations with a small training dataset also increases the risk of overfitting. The model can become too specialized in capturing noise and outliers in the training data, leading to poor generalization to unseen data.

Data Quality: More iterations won't compensate for a fundamentally small or poor-quality training dataset. If the dataset lacks representative examples or is biased, no amount of additional iterations can make up for these deficiencies.

Computation Time: Training a model with a large number of iterations can be computationally expensive. This may not be feasible in practice, especially for resource-constrained environments.

Validation Set Importance: When working with small datasets, the choice of the validation set becomes even more critical. A small validation set can lead to noisy performance estimates, making it challenging to determine whether the model is truly learning useful patterns or simply memorizing the training data.

In situations where you have a very small training dataset, it's essential to take steps to mitigate the risk of overfitting and make the most of the available data:

Use regularization techniques like L1 or L2 regularization to prevent overfitting.
Consider data augmentation if applicable to artificially increase the effective size of your training dataset.
Use cross-validation to assess how well your model generalizes to different subsets of your small dataset and to get a more reliable estimate of its performance.